Predicted strain

In [1]:
%run 01_Model_CFE.ipynb
from IPython.display import display
from cameo.strain_design import pathway_prediction

c:\Users\Driton\anaconda3\envs\python3716\lib\site-packages\nbformat\__init__.py:92: MissingIDFieldWarning: Code cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


ADS: frdp_c --> AMOR + ppi_c
CPR1_CYP71AV1: AMOR + h_c + nadph_c + o2_c --> AAOH + h2o_c + nadp_c
ADH1: AAOH + nad_c --> AAld + h_c + nadh_c
ALDH1: AAld + h2o_c + nadp_c --> AA + h_c + nadph_c


In [2]:
predictor = pathway_prediction.PathwayPredictor(iMM904)

In [3]:
pathways1=predictor.run(product='(2E,6E)-farnesyl diphosphate',max_predictions=6) 

,equation,lower_bound,upper_bound
MNXR55919,2.0 diphosphate + squalene + NADP(+) <=> H(+) ...,-1000.0,1000.0


,equation,lower_bound,upper_bound
MNXR69185,geranyl diphosphate + isopentenyl diphosphate ...,-1000.0,1000.0


,equation,lower_bound,upper_bound
MNXR55435,diphosphate + geranylgeranyl diphosphate <=> (...,-1000.0,1000.0


,equation,lower_bound,upper_bound
MNXR8602,diphosphate + squalene + NADP(+) <=> H(+) + pr...,-1000.0,1000.0
MNXR59177,diphosphate + presqualene diphosphate <=> 2.0 ...,-1000.0,1000.0


No pathway could be predicted. Terminating pathway predictions.
None (infeasible).


In [4]:
print(pathways1)

Pathway No. 1MNXR55919, rhea:32295: 2.0 diphosphate + squalene + NADP(+) <=> H(+) + 2.0 (2E,6E)-farnesyl diphosphate + NADPHPathway No. 2MNXR69185, rhea:19361: geranyl diphosphate + isopentenyl diphosphate <=> diphosphate + (2E,6E)-farnesyl diphosphatePathway No. 3MNXR55435, rhea:17653: diphosphate + geranylgeranyl diphosphate <=> (2E,6E)-farnesyl diphosphate + isopentenyl diphosphatePathway No. 4MNXR8602, rhea:22232: diphosphate + squalene + NADP(+) <=> H(+) + presqualene diphosphate + NADPHMNXR59177, rhea:22672: diphosphate + presqualene diphosphate <=> 2.0 (2E,6E)-farnesyl diphosphate


Testing whether pathway 4 increases AA production or biomass yield  

In [5]:
from cobra.io import read_sbml_model
from cobra import Reaction, Metabolite

In [6]:
PS_iMM904=iMM904.copy()


In [7]:
presqualene_diphosphate = Metabolite(id = 'psqldp', compartment='c', name='presqualene_diphosphate', formula='C30H52O7P2')


In [8]:
presqualene_diphosphate_synthase=Reaction('PSPPS')
presqualene_diphosphate_synthase_FDF=Reaction('r0575')

In [9]:
presqualene_diphosphate_synthase.add_metabolites(({PS_iMM904.metabolites.sql_c:-1,PS_iMM904.metabolites.ppi_c:-1,PS_iMM904.metabolites.nadp_c:-1,PS_iMM904.metabolites.h_c:1,presqualene_diphosphate:1,PS_iMM904.metabolites.nadph_c:1}))

presqualene_diphosphate_synthase_FDF.add_metabolites(({PS_iMM904.metabolites.ppi_c:-1,presqualene_diphosphate:-1,PS_iMM904.metabolites.frdp_c:2}))


In [10]:
PS_iMM904.add_reactions([presqualene_diphosphate_synthase])
PS_iMM904.add_reactions([presqualene_diphosphate_synthase_FDF])

In [11]:
# Make a copy of the medium dictionary
YEPDmediumSuc = PS_iMM904.medium.copy()

# Optimize the model for maximum biomass productivity
with PS_iMM904:
    # Define YEPDmedium
    YEPDmediumSuc["EX_glc__D_e"] = 0
    YEPDmediumSuc["EX_sucr_e"] = 20
    YEPDmediumSuc['EX_ala__L_e'] = 10
    YEPDmediumSuc['EX_arg__L_e'] = 10
    YEPDmediumSuc['EX_asp__L_e'] = 10
    YEPDmediumSuc['EX_cys__L_e'] = 10
    YEPDmediumSuc['EX_glu__L_e'] = 10
    YEPDmediumSuc['EX_gly_e'] = 10
    YEPDmediumSuc['EX_his__L_e'] = 10
    YEPDmediumSuc['EX_ile__L_e'] = 10
    YEPDmediumSuc['EX_leu__L_e'] = 10
    YEPDmediumSuc['EX_lys__L_e'] = 10
    YEPDmediumSuc['EX_met__L_e'] = 10
    YEPDmediumSuc['EX_phe__L_e'] = 10
    YEPDmediumSuc['EX_pro__L_e'] = 10
    YEPDmediumSuc['EX_ser__L_e'] = 10
    YEPDmediumSuc['EX_thr__L_e'] = 10
    YEPDmediumSuc['EX_trp__L_e'] = 10
    YEPDmediumSuc['EX_tyr__L_e'] = 10
    YEPDmediumSuc['EX_val__L_e'] = 10
    YEPDmediumSuc['EX_fe2_e'] = 10
    # Set YEPDmedium as the medium for the model
    PS_iMM904.medium = YEPDmediumSuc
    solutionSuc = PS_iMM904.optimize()
    print('Maximum theoretical biomass productivity:', solutionSuc.fluxes['BIOMASS_SC5_notrace'], '/h')

    # Change the objective function to ALDH1 and optimize again
    PS_iMM904.objective = PS_iMM904.reactions.ALDH1
    AA_productionSuc = PS_iMM904.optimize().objective_value
    print('Maximum theoretical productivity of Aldehyde_dehydrogenase_1 reaction:', AA_productionSuc, 'mmol/gDW*h')

Maximum theoretical biomass productivity: 1.3074654220045194 /h
Maximum theoretical productivity of Aldehyde_dehydrogenase_1 reaction: 2.000000000000001 mmol/gDW*h


No change in productivity or biomass  yield is seen from adding pathway 4 to the model. The comparison was to the yields seen in YEPD succrose in 03_Media_optimization.ipynb